In [1]:
!nvidia-smi

Wed Jul 19 10:39:02 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 108.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 15.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 70.4 MB/s eta 0:00

In [3]:
import pandas as pd
import nltk
import torch
import matplotlib.pyplot as plt

from transformers import pipeline, set_seed, AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import DataCollatorForSeq2Seq
from transformers import TrainingArguments, Trainer
from datasets import load_dataset, load_from_disk, load_metric
from nltk.tokenize import sent_tokenize
from tqdm import tqdm

In [4]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [6]:
model_ckpt = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Code Explanation

1. **model_ckpt = "google/pegasus-cnn_dailymail"**: This line assigns a
string value "google/pegasus-cnn_dailymail" to the variable model_ckpt. This string represents the pre-trained checkpoint or model name for the PEGASUS model trained on the CNN/Daily Mail dataset.

2. **tokenizer = AutoTokenizer.from_pretrained(model_ckpt)**: This line creates a tokenizer object by calling the from_pretrained method of the AutoTokenizer class. The AutoTokenizer class automatically selects an appropriate tokenizer based on the model checkpoint name (model_ckpt) provided. In this case, it will use the tokenizer specific to the PEGASUS model.

3. **model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)**: This line loads the pre-trained PEGASUS model for sequence-to-sequence language generation. It uses the AutoModelForSeq2SeqLM class and its from_pretrained method to load the model checkpoint specified by model_ckpt. The loaded model is then moved to a specified device (e.g., GPU) using the .to(device) method, which enables utilizing hardware acceleration if available.

In [7]:
# Downloading the Data
!wget https://github.com/alfandysurya2/project_datasets/raw/main/TextSummarizerDatasets/summarizer-data.zip
!unzip summarizer-data.zip

--2023-07-19 10:40:50--  https://github.com/alfandysurya2/project_datasets/raw/main/TextSummarizerDatasets/summarizer-data.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/alfandysurya2/project_datasets/main/TextSummarizerDatasets/summarizer-data.zip [following]
--2023-07-19 10:40:51--  https://raw.githubusercontent.com/alfandysurya2/project_datasets/main/TextSummarizerDatasets/summarizer-data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7903594 (7.5M) [application/zip]
Saving to: ‘summarizer-data.zip’

summarizer-data.zip 100%[===================>]   7.54M  --.-KB/s    in 0.03s   

In [8]:
dataset_samsum = load_from_disk('samsum_dataset')
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [9]:
split_lengths = [len(dataset_samsum[split]) for split in dataset_samsum]

print(f"split lengths: {split_lengths}")
print(f"features: {dataset_samsum['train'].column_names}")
print("\nDialogue:")

print(dataset_samsum['test'][1]['dialogue'])
print("\nSummary:")

print(dataset_samsum['test'][1]['summary'])

split lengths: [14732, 819, 818]
features: ['id', 'dialogue', 'summary']

Dialogue:
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I'll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)

Summary:
Eric and Rob are going to watch a stand-up on youtube.


## **Function**

### Code Explanation: convert_examples_to_features

1. **input_encodings = tokenizer(data['dialogue'], max_length=1024, truncation=True)**: This line uses the tokenizer object to tokenize and encode the dialogue data from the data dictionary. It passes the dialogue text as the input to the tokenizer's __call__ method. The max_length parameter specifies the maximum length allowed for the input sequence, and truncation=True indicates that any input sequence longer than the specified maximum length should be truncated. The resulting tokenized and encoded data is stored in the input_encodings variable.

2. **with tokenizer.as_target_tokenizer():**: This line indicates that the subsequent block of code should be executed with the tokenizer acting as a target tokenizer. In some tokenizers, this context manager allows you to specify different behavior for tokenizing and encoding target-specific sequences, such as summaries or labels.

3. **target_encodings = tokenizer(data['summary'], max_length=128, truncation=True)**: Within the context manager, this line tokenizes and encodes the summary data from the data dictionary. It uses the same tokenizer but with target-specific settings. The max_length parameter limits the maximum length of the target sequence, and truncation=True ensures that any sequence longer than the specified maximum length is truncated. The resulting tokenized and encoded data is stored in the target_encodings variable.

4. **The function then returns a dictionary containing the processed data**:
  - **input_ids**: input_encodings['input_ids'] retrieves the token IDs of the input sequence and assigns them to the 'input_ids' key in the returned dictionary.
  - **attention_mask**: input_encodings['attention_mask'] retrieves the attention mask of the input sequence (indicating which tokens are important) and assigns it to the 'attention_mask' key.
  - **labels**: target_encodings['input_ids'] retrieves the token IDs of the target sequence (summary or label) and assigns them to the 'labels' key in the returned dictionary.

In [10]:
def convert_examples_to_features(data):
  input_encodings = tokenizer(data['dialogue'], max_length=1024, truncation=True)

  with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(data['summary'], max_length = 128, truncation=True)

  return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }

### Code Explanation: generate_batch_sized_chunks

This Python code defines a function called generate_batch_sized_chunks that is used to split a list of elements into smaller batches of a specified size (batch_size). The purpose of this function is to process large datasets in smaller, manageable portions, allowing for more efficient memory usage and potential parallel processing.

Let's break down the code step by step:

1. **Function signature**: The function generate_batch_sized_chunks takes two parameters as input:

  - **list_of_elements**: This is the list of elements that you want to split into batches.
  - **batch_size**: This is an integer that represents the desired size of each batch.
2. **Generator function**: The function uses the yield statement, which means it is a generator function. A generator allows you to generate a sequence of values one at a time instead of computing and storing all values at once, which can be memory-intensive for large datasets.

3. **Looping through the list**: The function uses a for loop to iterate over the list_of_elements starting from index 0 and proceeding in increments of batch_size. The loop condition range(0, len(list_of_elements), batch_size) generates a sequence of indices corresponding to the start of each batch.

4. **Yielding batches**: Inside the loop, the yield statement is used to produce successive batches. The expression list_of_elements[i : i + batch_size] is a slice operation that extracts a portion of the list starting from index i and ending at index i + batch_size - 1. This creates a batch of elements of size batch_size.

5. **End of the function**: The function continues looping until it has processed all the elements in the list_of_elements. It will stop once the index i goes beyond the last index of the list.

In [11]:
def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

### Code Explanation: calculate_metric_on_test_ds

This code defines a function called calculate_metric_on_test_ds, which is used to evaluate a language model's performance using a specified metric on a test dataset. The function is specifically designed to calculate the `ROUGE (Recall-Oriented Understudy for Gisting Evaluation)` metric, commonly used for evaluating text summarization tasks. The function takes several arguments, including the dataset, the model, the tokenizer, and other optional parameters for batch processing and device placement.

Let's break down the code step by step:

1. **Function signature**: The function calculate_metric_on_test_ds takes several parameters as input:

     - **dataset**: This is the test dataset containing the articles and their corresponding reference summaries.
     - **metric**: This is an instance of a ROUGE metric class (not shown in the provided code), which is used to compute the ROUGE scores.
     - **model**: This is the pre-trained language model used for generating summaries.
     - **tokenizer**: This is the tokenizer associated with the model, used to preprocess the data for input to the model.
     - **batch_size**: An optional parameter that defines the size of each batch for processing (default value is 16).
     - **device**: An optional parameter specifying the device (e.g., "cpu" or "cuda") on which to perform model inference (default value is defined elsewhere).
     - **column_text**: The name of the column in the dataset containing the full articles (default value is "article").
     - **column_summary**: The name of the column in the dataset containing the reference summaries (default value is "highlights").

2. **Splitting the dataset into batches**: The function first divides the dataset into smaller batches of size batch_size using the generate_batch_sized_chunks function. It splits the dataset into article batches (article_batches) and target (reference summary) batches (target_batches). The generate_batch_sized_chunks function was not provided, but its purpose is to generate batches from a given list, which is used here for processing the data in smaller chunks.

3. **Model inference and evaluation**: The function then iterates through each batch of articles and their corresponding reference summaries. For each batch, it performs the following steps:

4. **Tokenization**: It uses the tokenizer to preprocess the articles and create input tensors suitable for the model. The tokenizer function converts the text to token IDs, pads/truncates the sequences to a maximum length of 1024 tokens (adjustable), and returns PyTorch tensors representing the input data (inputs).

5. **Summary generation**: It uses the pre-trained model to generate summaries for the input articles. The generate function of the model takes input_ids and attention_mask tensors, and it generates summaries using beam search with a length penalty and a maximum length of 128 tokens (adjustable).
Decoding: The generated summary tokens are decoded back into human-readable text using the tokenizer, removing special tokens and cleaning up tokenization spaces.

6. **Adding predictions and references to the metric**: The decoded summaries and the reference target summaries are added to the metric, which will be used to compute ROUGE scores.

7. **ROUGE computation and return**: After processing all batches, the function computes the ROUGE scores using the accumulated predictions and references in the metric object. The computed scores are then returned.

*Please note that the implementation assumes the existence of a ROUGE metric class, which was not provided in the code snippet. This class should be responsible for calculating the ROUGE scores based on predictions and references added to it, as shown in the code.*

In [12]:
def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

        # Finally, we decode the generated texts,
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score

## Data Preparation

In [13]:
dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features, batched=True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3635: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [14]:
dataset_samsum_pt['train']

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

In [19]:
dataset_samsum_pt['train'][1]

{'id': '13728867',
 'dialogue': 'Olivia: Who are you voting for in this election? \r\nOliver: Liberals as always.\r\nOlivia: Me too!!\r\nOliver: Great',
 'summary': 'Olivia and Olivier are voting for liberals in this election. ',
 'input_ids': [18038,
  151,
  2632,
  127,
  119,
  6228,
  118,
  115,
  136,
  2974,
  152,
  10463,
  151,
  35884,
  130,
  329,
  107,
  18038,
  151,
  2587,
  314,
  1242,
  10463,
  151,
  1509,
  1],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [18038, 111, 34296, 127, 6228, 118, 33195, 115, 136, 2974, 107, 1]}

## Training

In [16]:
#Training
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

trainer_args = TrainingArguments(
    output_dir = 'pegasus-samsum', num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=10,
    evaluation_strategy = 'steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
)

trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt['test'],
                  eval_dataset=dataset_samsum_pt['validation'])

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a PegasusTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


TrainOutput(global_step=51, training_loss=3.0761698928533816, metrics={'train_runtime': 153.2669, 'train_samples_per_second': 5.344, 'train_steps_per_second': 0.333, 'total_flos': 313317832187904.0, 'train_loss': 3.0761698928533816, 'epoch': 1.0})

In [17]:
# Evaluation
rouge_names = ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']
rouge_metric = load_metric('rouge')

score = calculate_metric_on_test_ds(
    dataset_samsum['test'][0:10],
    rouge_metric,
    trainer.model,
    tokenizer,
    batch_size=2,
    column_text='dialogue',
    column_summary= 'summary'
)

rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame(rouge_dict, index=[f'pegasus'])

<ipython-input-17-5482d50942a7>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric('rouge')


100%|██████████| 5/5 [00:16<00:00,  3.35s/it]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.017471,0.0,0.01512,0.014961


In [20]:
# Save the Model
model_pegasus.save_pretrained('pegasus_samsum_model')

In [21]:
tokenizer.save_pretrained('tokenizer')

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [22]:
# Load
tokenizer = AutoTokenizer.from_pretrained("/content/tokenizer")

In [24]:
#Prediction
gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}
sample_text = dataset_samsum["test"][0]["dialogue"]
reference = dataset_samsum["test"][0]["summary"]
pipe = pipeline("summarization", model="pegasus_samsum_model",tokenizer=tokenizer)

##
print("Dialogue:")
print(sample_text)

print("\nReference Summary:")
print(reference)

print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

Your max_length is set to 128, but your input_length is only 122. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)


Dialogue:
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye

Reference Summary:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.

Model Summary:
Amanda: Ask Larry Amanda: He called her last time we were at the park together .<n>Hannah: I'd rather you texted him .<n>Amanda: Just text him .
